In [1]:
# Import torch & Check CUDA availability
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [2]:
# Get CUDA device name
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

NVIDIA A30


#### Import Reddit

In [3]:
from torch_geometric.datasets import Reddit
import torch_geometric.transforms as T

# Import dataset from PyTorch Geometric
dataset = Reddit(root="/dfs6/pub/seminl1/Reddit", transform=T.ToSparseTensor())
data = dataset[0]
data = data.pin_memory()

# Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

Dataset: Reddit()
-------------------
Number of graphs: 1
Number of nodes: 232965
Number of features: 602
Number of classes: 41

Graph:
------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False


#### Graph Information

In [4]:
# Print first element
print(f'Graph: {data}')

Graph: Data(x=[232965, 602], y=[232965], train_mask=[232965], val_mask=[232965], test_mask=[232965], adj_t=[232965, 232965, nnz=114615892])


In [5]:
# Node feature matrix information
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([232965, 602])
tensor([[ 1.2334,  9.0430, -0.9233,  ..., -0.2579,  0.3112, -0.3772],
        [-0.1386, -0.2022,  0.1277,  ...,  0.1563,  0.1048, -0.6534],
        [-0.1330, -0.1962, -0.0296,  ...,  0.0358,  0.2864,  0.2744],
        ...,
        [-0.0614, -0.2022,  0.9698,  ...,  1.1064, -1.4323, -0.2398],
        [-0.1606, -0.2022, -0.0892,  ...,  0.7440, -0.5046, -2.2288],
        [ 0.0929,  0.2822,  0.1768,  ...,  0.2196,  0.5967,  0.5588]])


In [6]:
# Adjacency matrix for the edges
print(data.adj_t)

SparseTensor(row=tensor([     0,      0,      0,  ..., 232964, 232964, 232964]),
             col=tensor([   242,    249,    524,  ..., 231806, 232594, 232634]),
             size=(232965, 232965), nnz=114615892, density=0.21%)


In [7]:
# Ground-truth labels
print(f'y = {data.y.shape}')
print(data.y)

y = torch.Size([232965])
tensor([30, 17, 18,  ...,  3, 13, 13])


#### Sigle-layer GraphSAGE

In [8]:
# Create a simple GraphSAGE with only one GraphSAGE layer
import torch.nn.functional as F

from torch.nn import Linear
from torch_geometric.nn import SAGEConv

class GraphSAGE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.sage1 = SAGEConv(dataset.num_features, dataset.num_classes)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.02)

    def forward(self, x, adj_t):
        x = self.sage1(x, adj_t)
        z = F.log_softmax(x, dim=1)
        return x, z

#### Use mini-batch

In [9]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, train_loader):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 10

    model.train()
    for epoch in range(epochs):
        # Training on batches
        for batch in train_loader:
            batch = batch.to('cuda:0', non_blocking=True)
            optimizer.zero_grad()
            h, out = model(batch.x, batch.adj_t)
            loss = criterion(out, batch.y)
            loss.backward()
            optimizer.step()

    return model, h, out

##### num_workers = 16

In [10]:
from torch_geometric.loader import NeighborLoader

# NeighborLoader
train_loader = NeighborLoader(
    data,
    num_neighbors=[5],
    batch_size=8192,
    pin_memory=True,
    num_workers=16,
)

In [11]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GraphSAGE model
graphsage = GraphSAGE()
print(graphsage)
print()

# Train
start.record()
graphsage_model, graphsage_output, final_output = train(graphsage.to('cuda:0', non_blocking=True), train_loader)
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (10 Epochs):', elapsed_time*0.001, 'seconds')

GraphSAGE(
  (sage1): SAGEConv(602, 41, aggr=mean)
)

Elapsed Time (10 Epochs): 13.3694990234375 seconds


##### num_workers = 8

In [12]:
from torch_geometric.loader import NeighborLoader

# NeighborLoader
train_loader = NeighborLoader(
    data,
    num_neighbors=[5],
    batch_size=8192,
    pin_memory=True,
    num_workers=8,
)

In [13]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GraphSAGE model
graphsage = GraphSAGE()
print(graphsage)
print()

# Train
start.record()
graphsage_model, graphsage_output, final_output = train(graphsage.to('cuda:0', non_blocking=True), train_loader)
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (10 Epochs):', elapsed_time*0.001, 'seconds')

GraphSAGE(
  (sage1): SAGEConv(602, 41, aggr=mean)
)

Elapsed Time (10 Epochs): 12.0540029296875 seconds


##### num_workers = 4

In [14]:
from torch_geometric.loader import NeighborLoader

# NeighborLoader
train_loader = NeighborLoader(
    data,
    num_neighbors=[5],
    batch_size=8192,
    pin_memory=True,
    num_workers=4,
)

In [15]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GraphSAGE model
graphsage = GraphSAGE()
print(graphsage)
print()

# Train
start.record()
graphsage_model, graphsage_output, final_output = train(graphsage.to('cuda:0', non_blocking=True), train_loader)
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (10 Epochs):', elapsed_time*0.001, 'seconds')

GraphSAGE(
  (sage1): SAGEConv(602, 41, aggr=mean)
)

Elapsed Time (10 Epochs): 12.430359375 seconds


##### num_workers = 2

In [16]:
from torch_geometric.loader import NeighborLoader

# NeighborLoader
train_loader = NeighborLoader(
    data,
    num_neighbors=[5],
    batch_size=8192,
    pin_memory=True,
    num_workers=2,
)

In [17]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GraphSAGE model
graphsage = GraphSAGE()
print(graphsage)
print()

# Train
start.record()
graphsage_model, graphsage_output, final_output = train(graphsage.to('cuda:0', non_blocking=True), train_loader)
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (10 Epochs):', elapsed_time*0.001, 'seconds')

GraphSAGE(
  (sage1): SAGEConv(602, 41, aggr=mean)
)

Elapsed Time (10 Epochs): 12.720403320312501 seconds


##### num_workers = 0

In [18]:
from torch_geometric.loader import NeighborLoader

# NeighborLoader
train_loader = NeighborLoader(
    data,
    num_neighbors=[5],
    batch_size=8192,
    pin_memory=True,
)

In [19]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GraphSAGE model
graphsage = GraphSAGE()
print(graphsage)
print()

# Train
start.record()
graphsage_model, graphsage_output, final_output = train(graphsage.to('cuda:0', non_blocking=True), train_loader)
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (10 Epochs):', elapsed_time*0.001, 'seconds')

GraphSAGE(
  (sage1): SAGEConv(602, 41, aggr=mean)
)

Elapsed Time (10 Epochs): 4.46794970703125 seconds
